# BEGAN을 이용한 이미지생성

****라이브러리 읽어들이기****

In [ ]:
# https://drive.google.com/uc?export=download&id=1URxZOJTO38qb3v1hOT3usuAE7nTGQDnL 참조

import os

import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import losses
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, Activation, Flatten, Dense, UpSampling2D, Reshape, Lambda, Input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import img_to_array, array_to_img
import tensorflow as tf
print(tf.__version__)

**이미지를 저장하는 함수**


In [75]:
def save_imgs(path, imgs, rows, cols):
    """이미지를 타일 형태로 저장
    
    Arguments:
        path (str): 저장할 폴더 경로
        imgs (np.array): 저장할 이미지 리스트
        rows (int): 타일의 세로 크기
        cols (int): 타일의 가로 크기
        
    Returns:
        None
    """
    base_width = imgs.shape[1]
    base_height = imgs.shape[2]
    channels = imgs.shape[3]
    output_shape = (
        base_height*rows,
        base_width*cols,
        channels
    )
    buffer = np.zeros(output_shape)
    for row in range(rows):
        for col in range(cols):
            img = imgs[row*cols + col]
            buffer[
                row*base_height:(row + 1)*base_height,
                col*base_width:(col + 1)*base_width
            ] = img
    array_to_img(buffer).save(path)

**이미지 데이터 읽어 들이기**

In [89]:
DATA_DIR = 'data/'
BATCH_SIZE = 24
IMG_SHAPE = (28, 28, 3)

data_gen = ImageDataGenerator(rescale=1/255.)
train_data_generator = data_gen.flow_from_directory(
    directory=DATA_DIR,
    classes=['mnist'],
    class_mode=None,
    batch_size=BATCH_SIZE,
    target_size=IMG_SHAPE[:2]
)

Found 600 images belonging to 1 classes.


**Encoder 정의**

In [90]:
def build_encoder(input_shape, z_size, n_filters, n_layers):
    """Encoder구축
    
    Arguments:
        input_shape (int): 이미지의 shape
        z_size (int): 특징 공간의 차원 수
        n_filters (int): 파일 수
        
    Returns:
        model (Model): 인코더 모델 
    """
    model = Sequential()
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            input_shape=input_shape,
            padding='same'
        )
    )
    model.add(Conv2D(n_filters, 3, padding='same'))
    for i in range(2, n_layers + 1):
        model.add(
            Conv2D(
                i*n_filters,
                3,
                activation='elu',
                padding='same'
            )
        )
        model.add(
                Conv2D(
                i*n_filters,
                3,
                activation='elu',
                strides=2,
                padding='same'
            )
        )
    model.add(Conv2D(n_layers*n_filters, 3, padding='same'))
    model.add(Flatten())
    model.add(Dense(z_size))
    
    return model

**생성자/Decoder 정의**

In [91]:
def build_decoder(output_shape, z_size, n_filters, n_layers):
    """Decoder 구축
    
    Arguments:
        output_shape (np.array): 이미지 shape
        z_size (int): 특징 공간의 차원 수
        n_filters (int): 파일 수
        n_layers (int): 레이어 수
        
    Returns:
        model (Model): 디코더 모델 
    """
    # UpSampling2D로 몇 배로 확대할지 계산
    scale = 2**(n_layers - 1)
    # 합성곱층의 처음 입력 사이즈를 scale로부터 역산
    fc_shape = (
        output_shape[0]//scale,
        output_shape[1]//scale,
        n_filters
    )
    # 완전연결 계층에서 필요한 사이즈를 역산
    fc_size = fc_shape[0]*fc_shape[1]*fc_shape[2]
    
    model = Sequential()
    # 완전연결 계층
    model.add(Dense(fc_size, input_shape=(z_size,)))
    model.add(Reshape(fc_shape))
    
    # 합성곱층 반복
    for i in range(n_layers - 1):
        model.add(
            Conv2D(
                n_filters,
                3,
                activation='elu',
                padding='same'
            )
        )
        model.add(
            Conv2D(
                n_filters,
                3,
                activation='elu',
                padding='same'
            )
        )
        model.add(UpSampling2D())
        
    # 마지막 층은 UpSampling2D가 불필요
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    model.add(
        Conv2D(
            n_filters,
            3,
            activation='elu',
            padding='same'
        )
    )
    # 출력층에서는 3채널로
    model.add(Conv2D(3, 3, padding='same'))
    
    return model

**생성자 정의**

In [92]:
def build_generator(img_shape, z_size, n_filters, n_layers):
    decoder = build_decoder(
        img_shape, z_size, n_filters, n_layers
    )
    return decoder

**구분자 정의**

In [93]:
def build_discriminator(img_shape, z_size, n_filters, n_layers):
    encoder = build_encoder(
        img_shape, z_size, n_filters, n_layers
    )
    decoder = build_decoder(
        img_shape, z_size, n_filters, n_layers
    )
    return keras.models.Sequential((encoder, decoder))

**구분자의 학습용 네트워크**

In [94]:
def build_discriminator_trainer(discriminator):
    img_shape = discriminator.input_shape[1:]
    real_inputs = Input(img_shape)
    fake_inputs = Input(img_shape)
    real_outputs = discriminator(real_inputs)
    fake_outputs = discriminator(fake_inputs)

    return Model(
        inputs=[real_inputs, fake_inputs],
        outputs=[real_outputs, fake_outputs]
    )

**네트워크 구축**

In [95]:
n_filters = 64  #  필터 수
n_layers = 3 # 레이어 수
z_size = 32  #  특징 공간의 차원

generator = build_generator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator = build_discriminator(
    IMG_SHAPE, z_size, n_filters, n_layers
)
discriminator_trainer = build_discriminator_trainer(
    discriminator
)

generator.summary()
# discriminator.layers[1]은 디코더를 나타냄
discriminator.layers[1].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 3136)              103488    
_________________________________________________________________
reshape_21 (Reshape)         (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_289 (Conv2D)          (None, 7, 7, 64)          36928     
_________________________________________________________________
conv2d_290 (Conv2D)          (None, 7, 7, 64)          36928     
_________________________________________________________________
up_sampling2d_67 (UpSampling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_291 (Conv2D)          (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_292 (Conv2D)          (None, 14, 14, 64)        36928     
__________

**손실 함수 정의**

In [96]:
from tensorflow.python.keras.losses import mean_absolute_error


def build_generator_loss(discriminator):
    # discriminator를 사용해서 손실 함수 정의
    def loss(y_true, y_pred):
        # y_true 
        reconst = discriminator(y_pred)
        return mean_absolute_error(
            reconst,
            y_pred
        )
    return loss

**generator 컴파일**

In [97]:
# 초기 학습률(Generator)
g_lr = 0.0001

generator_loss = build_generator_loss(discriminator)
generator.compile(
    loss=generator_loss,
    optimizer=Adam(g_lr)
)

**구분자 컴파일**

In [98]:
# 초기 학습률(Discriminator)
d_lr = 0.0001

# k_var는 수치(일반 변수)
k_var = 0.0
# k : Keras(TensorFlow) Variable
k = K.variable(k_var)
discriminator_trainer.compile(
    loss=[
        mean_absolute_error,
        mean_absolute_error
    ],
    loss_weights=[1., -k],
    optimizer=Adam(d_lr)
)

**수렴 판정용 함수 정의**

In [99]:
def measure(real_loss, fake_loss, gamma):
    return real_loss + np.abs(gamma*real_loss - fake_loss)

**학습 코드**

In [ ]:
# k의 갱신에 이용할 파라미터
GAMMA = 0.5
LR_K = 0.001

# 반복 수. 100000～1000000 정도로 지정
TOTAL_STEPS = 10

# 모델과 확인용 생성 이미지를 저장할 폴더

IMG_SAVE_DIR = 'imgs'
# 확인용으로 4x4 개의 이미지를 생성
IMG_SAMPLE_SHAPE = (4, 4)
N_IMG_SAMPLES = np.prod(IMG_SAMPLE_SHAPE)


# 저장할 폴더가 없다면 생성
os.makedirs(IMG_SAVE_DIR, exist_ok=True)

# 샘플이미지용 랜덤 시드
sample_seeds = np.random.uniform(
    -1, 1, (N_IMG_SAMPLES, z_size)
)

history = []
logs = []

for step, batch in enumerate(train_data_generator):
  

    # 임의의 값 생성
    z_g = np.random.uniform(
        -1, 1, (BATCH_SIZE, z_size)
    )
    z_d = np.random.uniform(
        -1, 1, (BATCH_SIZE, z_size)
    )
    
    # 생성 이미지(구분자의 학습에 이용)
    g_pred = generator.predict(z_d)
    
    # 생성자를 1스텝 학습시킨다
    generator.train_on_batch(z_g, batch)
    # 구분자를 1스텝 학습시킨다
    _, real_loss, fake_loss = discriminator_trainer.train_on_batch(
            [batch, g_pred],
            [batch, g_pred]
    )

    # k 를 갱신
    k_var += LR_K*(GAMMA*real_loss - fake_loss)
    K.set_value(k, k_var)
    

    # g_measure 을 계산하기 위한 loss 저장
    history.append({
        'real_loss': real_loss,
        'fake_loss': fake_loss
    })

    # 10번에 1번씩 로그 표시
    if step%10 == 0:
        # 과거 10 번의 measure 의 평균
        measurement = np.mean([
            measure(
                loss['real_loss'],
                loss['fake_loss'],
                GAMMA
            )
            for loss in history[-10:]
        ])
        
        logs.append({
            'k': K.get_value(k),
            'measure': measurement,
            'real_loss': real_loss,
            'fake_loss': fake_loss
        })
        print(logs[-1])

        # 이미지 저장  
        img_path = '{}/generated_{}.png'.format(
            IMG_SAVE_DIR,
            step
        )
        save_imgs(
            img_path,
            generator.predict(sample_seeds),
            rows=IMG_SAMPLE_SHAPE[0],
            cols=IMG_SAMPLE_SHAPE[1]
        )
  

{'real_loss': 0.049274366, 'measure': 0.06939699733629823, 'k': 0.020042641, 'fake_loss': 0.004514552}
{'real_loss': 0.04165324, 'measure': 0.06225148574449122, 'k': 0.020221815, 'fake_loss': 0.004084554}
{'real_loss': 0.04597017, 'measure': 0.06296164328232408, 'k': 0.020401094, 'fake_loss': 0.006035212}
{'real_loss': 0.042247687, 'measure': 0.06434513765852898, 'k': 0.020588748, 'fake_loss': 0.003389097}
{'real_loss': 0.04659957, 'measure': 0.06238659273367375, 'k': 0.02076724, 'fake_loss': 0.0055764955}
{'real_loss': 0.045257103, 'measure': 0.06112694253679365, 'k': 0.020935792, 'fake_loss': 0.0041048005}
{'real_loss': 0.042053778, 'measure': 0.06467423134017736, 'k': 0.021122888, 'fake_loss': 0.0035600562}
{'real_loss': 0.046508964, 'measure': 0.060783356754109265, 'k': 0.021293417, 'fake_loss': 0.0056531336}
{'real_loss': 0.0381217, 'measure': 0.062064941576682034, 'k': 0.021467876, 'fake_loss': 0.005105612}
{'real_loss': 0.04963976, 'measure': 0.06320860176347196, 'k': 0.02164864